In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# 모델
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
## test data set 만드는 함수
def make_test_xy(wf,gf):
    gf.columns = ["time","m0","m1","m2","m3","m4"]
    train = pd.merge(gf,wf,on="time")
    train["time"] = pd.to_datetime(train["time"])
    train["month"]=train["time"].dt.month
    train["day"]=train["time"].dt.day
    train["hour"]=train["time"].dt.hour
    
    train_1 = train[(train["hour"]<=10) & (train["hour"]>=7)]
    train_2 = train[(train["hour"]<=14) & (train["hour"]>=11)]
    train_3 = train[(train["hour"]<=19) & (train["hour"]>=15)]
    
    train11 = train_1[["m0","m1","m2","m3","uv_idx","elevation"]]
    train12 = train_2[["m0","m1","m2","m3","uv_idx","elevation"]]
    train13 = train_3[["m0","m1","m2","m3","uv_idx","elevation"]]
    train21 = train_1.drop(["m0","m1","m2","m3","m4","uv_idx","elevation","time","hour"],axis=1)
    train22 = train_2.drop(["m0","m1","m2","m3","m4","uv_idx","elevation","time","hour"],axis=1)
    train23 = train_3.drop(["m0","m1","m2","m3","m4","uv_idx","elevation","time","hour"],axis=1)
    
    return train11,train12,train13,train21,train22,train23

#  예측 진행

In [9]:
#실제값 정의
real_dict = {"1025" : [0, 0, 0, 0, 0, 0,
                      0, 10, 31, 54, 70, 72, 78, 73, 63, 40, 20, 3, 0,
                      0, 0, 0, 0, 0],
             "1026" : [0, 0, 0, 0, 0, 0,
                      1, 3, 11, 40, 61, 52, 75, 61, 62, 40, 18, 2, 0,
                      0, 0, 0, 0, 0],
             "1027" : [0, 0, 0, 0, 0, 0,
                      1, 9, 36, 46, 53, 74, 89, 81, 64, 54, 23, 4, 0,
                      0, 0, 0, 0, 0],
             "1028" : [0, 0, 0, 0, 0, 0,
                      0, 5, 29, 56, 67, 82, 78, 72, 63, 46, 17, 3, 0,
                      0, 0, 0, 0, 0],
             "1029" : [0, 0, 0, 0, 0, 0,
                      0, 4, 20, 56, 66, 76, 53, 57, 66, 42, 20, 2, 0,
                      0, 0, 0, 0, 0],
             "1030" : [0, 0, 0, 0, 0, 0,
                      0, 4, 20, 56, 65, 79, 81, 75, 62, 45, 22, 3, 0,
                      0, 0, 0, 0, 0],
             "1031" : [0, 0, 0, 0, 0, 0,
                      0, 4, 8, 22, 37, 76, 73, 75, 58, 44, 22, 4, 0,
                      0, 0, 0, 0, 0],
             "1101" : [0, 0, 0, 0, 0, 0,
                       0, 6, 30, 51, 58, 68, 68, 69, 55, 38, 18, 1, 0,
                       0, 0, 0, 0, 0],
             "1102" : [0, 0, 0, 0, 0, 0,
                       0, 4, 32, 46, 71, 76, 79, 73, 64, 44, 22, 4, 0,
                       0, 0, 0, 0, 0]}

In [3]:
gens = pd.read_csv("../gens.csv")
gens["time"] = pd.to_datetime(gens["time"])
gens["month"] = gens["time"].dt.month

In [4]:
gens[gens["month"].isin([9,10,11])]

,time,amount,month
1775,2022-09-01 00:00:00+09:00,0.0,9
1776,2022-09-01 01:00:00+09:00,0.0,9
1777,2022-09-01 02:00:00+09:00,0.0,9
1778,2022-09-01 03:00:00+09:00,0.0,9
1779,2022-09-01 04:00:00+09:00,0.0,9
...,...,...,...
11611,2023-10-15 20:00:00+09:00,0.0,10
11612,2023-10-15 21:00:00+09:00,0.0,10
11613,2023-10-15 22:00:00+09:00,0.0,10
11614,2023-10-15 23:00:00+09:00,0.0,10


In [5]:
import seaborn as sns

In [6]:
# 예측에 사용할 데이터 셋 불러오기 
x_pred_ori = pd.read_csv("../data/gen_fcst_10_2023-10-31.csv")
x_pred_wf = pd.read_csv("../data/weather_forecasts/wf10_2023-10-31.csv")

In [10]:
day = real_dict["1031"]

In [11]:
# 함수에 넣어서 데이터셋 분리
test11,test12,test13,test21,test22,test23 = make_test_xy(x_pred_wf,x_pred_ori)

In [12]:
import joblib

In [13]:
model11=joblib.load("../model/시간1회귀fitting_season9-11_r1.pkl")
model12=joblib.load("../model/시간2회귀fitting_season9-11_r1.pkl")
model13=joblib.load("../model/시간3회귀fitting_season9-11_r1.pkl")
# 예측 진행 (1차 모델링 y_hat)
y_pred11=model11.predict(test11)
y_pred12=model12.predict(test12)
y_pred13=model13.predict(test13)
model21=joblib.load("../model/시간1잔차fitting_season9-11_r1.pkl")
model22=joblib.load("../model/시간2잔차fitting_season9-11_r1.pkl")
model23=joblib.load("../model/시간3잔차fitting_season9-11_r1.pkl")
y_pred21 = model21.predict(test21)
y_pred22 = model22.predict(test22)
y_pred23 = model23.predict(test23)
result1 = y_pred11+y_pred21
result2 = y_pred12+y_pred22
result3 = y_pred13+y_pred23
result = np.concatenate((result1,result2,result3))
res1 = result

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [14]:
res1

array([-5.76939092e-02,  6.44917541e+00,  2.63923686e+01,  5.16400589e+01,
        6.90145273e+01,  7.82816194e+01,  7.50092600e+01,  7.18578439e+01,
        5.73817330e+01,  3.75224476e+01,  2.06311356e+01,  1.84776832e+00,
        2.79279719e-01])

In [15]:
# 예측에 사용할 데이터 셋 불러오기 
x_pred_ori = pd.read_csv("../data/gen_fcst_17_2023-10-31.csv")
x_pred_wf = pd.read_csv("../data/weather_forecasts/wf17_2023-10-31.csv")
# 함수에 넣어서 데이터셋 분리
test11,test12,test13,test21,test22,test23 = make_test_xy(x_pred_wf,x_pred_ori)

In [17]:
model11=joblib.load("../model/시간1회귀fitting_season9-11_r2.pkl")
model12=joblib.load("../model/시간2회귀fitting_season9-11_r2.pkl")
model13=joblib.load("../model/시간3회귀fitting_season9-11_r2.pkl")
# 예측 진행 (1차 모델링 y_hat)
y_pred11=model11.predict(test11)
y_pred12=model12.predict(test12)
y_pred13=model13.predict(test13)
model21=joblib.load("../model/시간1잔차fitting_season9-11_r2.pkl")
model22=joblib.load("../model/시간2잔차fitting_season9-11_r2.pkl")
model23=joblib.load("../model/시간3잔차fitting_season9-11_r2.pkl")
y_pred21 = model21.predict(test21)
y_pred22 = model22.predict(test22)
y_pred23 = model23.predict(test23)
result1 = y_pred11+y_pred21
result2 = y_pred12+y_pred22
result3 = y_pred13+y_pred23
result = np.concatenate((result1,result2,result3))
res2 = result

c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: '../model/시간3잔차fitting_season9-11_r2.pkl'

In [103]:
res2

array([ 0.96427595,  7.39847258, 28.64790555, 52.93967752, 67.07920672,
       76.47278824, 74.97701908, 72.91373648, 55.57736281, 36.73789272,
       16.56069981,  0.08620579, -1.72706062])

In [104]:
pred = (res1+res2)/2

In [106]:
pred = [0,0,0,0,0,0, 
        0.45329102,  6.923824  , 27.52013709, 52.28986821, 68.04686702,
       77.37720381, 74.99313956, 72.3857902 , 56.47954789, 37.13017018,
       18.59591772,  0.96698706, -0.72389045,
       0,0,0,0,0]

In [7]:
import numpy as np

def testMSEnIncentive(pred,day):
    realGens1030 = np.array(day)    
    
    rss = np.power(pred - realGens1030, 2)
    rss1 = rss[6:10]
    rss2 = rss[10:14]
    rss3 = rss[14:19]
    mse1 = sum(rss1/len(rss1))
    mse2 = sum(rss2/len(rss2))
    mse3 = sum(rss3/len(rss3))
    mse = [mse1,mse2,mse3]

    score_df = abs(pred - realGens1030)*100/99
    score = []
    for idx, i in enumerate(score_df):
        if i <=6:
            score.append(realGens1030[idx] * 4)
        elif (i <=8) & (i > 6):
            score.append(realGens1030[idx] * 3)
        else:
            score.append(0)
    score1 = score[6:10]
    score2 = score[10:14]
    score3 = score[14:19]
    final_score = [score1,score2,score3]
    
    sum_score = sum(score)

    return mse, final_score , score, sum_score

In [107]:
testMSEnIncentive(pred,day)

([326.8165219308377, 244.15283506897373, 14.163459484276665],
 [[0, 16, 0, 0], [0, 304, 292, 300], [232, 132, 88, 16, 0]],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  16,
  0,
  0,
  0,
  304,
  292,
  300,
  232,
  132,
  88,
  16,
  0,
  0,
  0,
  0,
  0,
  0],
 1380)

In [ ]:
day = real_dict["1029"]
testMSEnIncentive(pred,day)